In [1]:
import time
import pandas as pd
from pyspark.sql import SparkSession
from operator import add
import os
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import length
from pyspark.sql.functions import regexp_replace, lower
from pyspark.sql.functions import avg, max
import matplotlib.pyplot as plt
from pyspark.sql.functions import collect_list
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import StringIndexer
import pandas as pd
import numpy as np

In [2]:
spark_session = SparkSession.builder\
.appName("yufengchen_app")\
.config("spark.dynamicAllocation.enabled", True)\
.config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
.config("spark.shuffle.service.enabled", True)\
.config("spark.dynamicAllocation.executorIdleTimeout","30s")\
.config("spark.cores.max", 4)\
.config("spark.executor.memory", "8g") \
.getOrCreate()

spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")


file_path = '/home/ubuntu/Pre_processed_RC_2011-08/part-00000-f787f5a3-150d-4158-be90-c97961fb4e12-c000.json'

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/04 15:15:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# read the json
reddit_comments_df = spark_session.read.json(file_path)

In [4]:
author_indexer = StringIndexer(inputCol="author", outputCol="user_id")
reddit_comments_df = author_indexer.fit(reddit_comments_df).transform(reddit_comments_df)
link_indexer = StringIndexer(inputCol="link_id", outputCol="item_id")
reddit_comments_df = link_indexer.fit(reddit_comments_df).transform(reddit_comments_df)

In [5]:
# to int
reddit_comments_df = reddit_comments_df.withColumn("created_utc", reddit_comments_df["created_utc"].cast("int"))

In [6]:
(training_data, test_data) = reddit_comments_df.randomSplit([0.1, 0.9])

In [7]:
# ALS model
als = ALS(userCol="user_id", itemCol="item_id", ratingCol="score", nonnegative=True, implicitPrefs=False, coldStartStrategy="drop")
model = als.fit(training_data)

23/03/04 15:21:24 ERROR Instrumentation: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space

	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2454)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2402)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2402)
	at org.apache.spark.scheduler.DAGScheduler.submitMissingTasks(DAGScheduler.scala:1488)
	at org.apache.spark.scheduler.DAGScheduler.submitStage(DAGScheduler.scala:1307)
	at org.apache

Py4JJavaError: An error occurred while calling o142.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space

	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2454)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2402)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2402)
	at org.apache.spark.scheduler.DAGScheduler.submitMissingTasks(DAGScheduler.scala:1488)
	at org.apache.spark.scheduler.DAGScheduler.submitStage(DAGScheduler.scala:13

In [ ]:
# predict
predictions = model.transform(test_data)

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [ ]:
predictions = predictions.take(10)

In [ ]:
predicted_scores = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()
actual_scores = predictions.select("score").rdd.flatMap(lambda x: x).collect()

In [ ]:
plt.scatter(predicted_scores, actual_scores)
plt.xlabel("Predicted Score")
plt.ylabel("Actual Score")
plt.title("Collaborative Filtering Recommender Results")
plt.show()

In [ ]:
user_recs = model.recommendForAllUsers(5)
user_recs.show(truncate=False)

In [ ]:
item_recs = model.recommendForAllItems(5)
item_recs.show(truncate=False)

In [ ]:
spark.stop()